[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/Text-To-Video-Finetuning-colab/blob/main/dataset.ipynb)

In [ ]:
%cd /content

!pip install datasets yt_dlp scenedetect[opencv]

import datasets, json
from yt_dlp import YoutubeDL

!mkdir /content/videos

def download_video(id):
  ydl_opts = {'overwrites':True, 'format':'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4', 'outtmpl':f'/content/videos/{id}.mp4'}
  with YoutubeDL(ydl_opts) as ydl:
    ydl.download(id)
    return f"/content/videos/{id}.mp4"

dataset = datasets.load_dataset("camenduru/hdvila_test", split='train[:1%]')
# dataset = datasets.load_dataset("camenduru/microsoft-XPretrain", split='train[:1%]')
first_2_data = list(dataset)[:2]

for dump in first_2_data:
  json_string = json.dumps(dump)
  row = json.loads(json_string)
  video_id = row['video_id']
  print(video_id)
  try:
    download_video(video_id)
    video_path = f"/content/videos/{video_id}.mp4"
    # !scenedetect -i {video_path} -o /content/adaptive --drop-short-scenes detect-adaptive split-video
    !scenedetect -i {video_path} -o /content/content --drop-short-scenes detect-content split-video
    # !scenedetect -i {video_path} -o /content/threshold --drop-short-scenes detect-threshold split-video
  except:
    pass

In [ ]:
%cd /content

!pip install yt_dlp scenedetect[opencv]

import json
from yt_dlp import YoutubeDL

!mkdir /content/videos

def download_video(id):
  ydl_opts = {'overwrites':True, 'format':'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4', 'outtmpl':f'/content/videos/{id}.mp4'}
  with YoutubeDL(ydl_opts) as ydl:
    ydl.download(f"https://www.imdb.com/video/{id}")
    return f"/content/videos/{id}.mp4"

import requests, json
url_popular= "https://camenduru-imdb.hf.space/api/chart/popular"
response_popular = requests.get(url_popular)
if response_popular.status_code == 200:
  json_data_popular = response_popular.json()
  json_string_popular = json.dumps(json_data_popular)
  movies = json.loads(json_string_popular)['data']
  # movies = movies[2:] # skip first 2
  for movie in movies:
    url_titles = f"https://camenduru-imdb.hf.space/api/titles/{movie['titleId']}"
    response_titles = requests.get(url_titles)
    if response_titles.status_code == 200:
      json_data_titles = response_titles.json()
      json_string_titles = json.dumps(json_data_titles)
      videos = json.loads(json_string_titles)['data']['videos']
      my_array = []
      my_dict = {}
      for video in videos:
        if 'Trailer' in video['duration']:
          minutes, seconds = video['duration'].split()[1].split(":")
          minutes = int(minutes) * 60
          seconds = int(seconds)
          total_seconds = minutes + seconds
          my_array.append(total_seconds)
          my_dict[total_seconds] = video['id']
      download_video_id = my_dict.get(max(my_array))
      print(download_video_id)
      download_video(download_video_id)
      video_path = f"/content/videos/{download_video_id}.mp4"
      !scenedetect -i {video_path} -o /content/content --drop-short-scenes detect-content split-video save-images -n 1
    else:
      print("Error getting JSON data:", response.status_code)

else:
  print("Error getting JSON data:", response.status_code)

In [ ]:
import os

def find_top_10_largest_files(folder_path):
    file_sizes = []
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            file_size = os.path.getsize(file_path)
            file_sizes.append((file_path, file_size))

    sorted_files = sorted(file_sizes, key=lambda x: x[1], reverse=True)
    top_10_files = sorted_files[:10]
    return top_10_files

folder_path = "/content/content"
top_10_files = find_top_10_largest_files(folder_path)
for file_path, file_size in top_10_files:
    print(f"File: {file_path} | Size: {file_size} bytes")

In [ ]:
%cd /content

!git clone -b dev https://github.com/camenduru/Video-BLIP2-Preprocessor
%cd /content/Video-BLIP2-Preprocessor
!pip install -r requirements.txt

In [ ]:
!python preprocess.py --video_directory /content/content --config_name "My Videos" --config_save_name "my_videos"